In [1]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import numpy as np
from math import ceil
import copy

from simulation import run_simulation
from graph_create import ring_lattice, reduce_providers_simplest, powerlaw_dist_time
from sim_strategies import EvenAlloc, MWU
import util

init_notebook_mode(connected=True)

In [2]:
# Constants
num_vertices = 100
regularity = 4

# MWU parameters
lrate = 0.5

In [3]:
def create_rl(n, k, p, r):
    return ring_lattice(k, n, p, r)

In [4]:
def plot_util_comp(given_utils, r, regularity, diam):
    fig = go.Figure()
    
    social_utils, even_utils = given_utils

    #Add averages trace
    p_vals = [ p for (p, uts) in social_utils ]
    ut_vals = [ np.average(uts) for (p, uts) in social_utils ]
    fig.add_trace(go.Scatter(x=p_vals,
        y=ut_vals, mode='lines+markers', hoverinfo='skip',
        name='avg social welfare rate (MWU)', marker=dict(color='Blue')))
    
    even_p_vals = [ p for (p, uts) in even_utils ]
    even_ut_vals = [ np.average(uts) for (p, uts) in even_utils ]
    fig.add_trace(go.Scatter(x=even_p_vals,
        y=even_ut_vals, mode='lines+markers', hoverinfo='skip',
        name='avg social welfare rate (even alloc)',
        marker=dict(color='Orange')))
    
    #Add individual points trace
    indiv_p_vals = []
    indiv_ut_vals = []
    for (p, uts) in social_utils:
        indiv_ut_vals.extend(uts)
        indiv_p_vals.extend([p] * len(uts))
    fig.add_trace(go.Scatter(x=indiv_p_vals,
        y=indiv_ut_vals, mode='markers', hoverinfo='skip', opacity=0.7,
        marker=dict(color='LightSkyBlue'), name='social welfare per p'))

    even_indiv_p_vals = []
    even_indiv_ut_vals = []
    for (p, uts) in even_utils:
        even_indiv_ut_vals.extend(uts)
        even_indiv_p_vals.extend([p] * len(uts))
    fig.add_trace(go.Scatter(x=even_indiv_p_vals,
        y=even_indiv_ut_vals, mode='markers', hoverinfo='skip', opacity=0.7,
        marker=dict(color='rgba(244,204,102,1)'),
        name='social welfare per p (even alloc)'))


    fig.layout.update(showlegend=False)
    title_fmt = "Average Social Welfare (Normalized) vs p (r={0}, diam={1})"
    plot_title = title_fmt.format(r, diam)
    fig.layout.update(title=plot_title, showlegend=True,
        xaxis=dict(title='p'), yaxis=dict(title='avg social welfare (MWU)'),
        plot_bgcolor='rgba(0,0,0,0)')

    iplot(fig)

In [5]:
def comp_strat(num_vertices, regularity, graph_func, strat, strat_params, plaw_resources=False):
    for r in [ 2 ** k for k in range(1, 8) ][::-1]:
        social_utils = []
        even_utils = []
        for p in np.linspace(1, 0, 10, endpoint=False):
            utils = []
            eutils = []
            num_iter = 10
            for i in range(num_iter):
                G = graph_func(num_vertices, regularity, p, r)

                graph_diam = util.calc_diameter(G)

                reduce_providers_simplest(G)
                
                if plaw_resources:
                    powerlaw_dist_time(G, 2)
                    
                #Make copy of graph to run even alloc strategy on
                G_even = copy.deepcopy(G)

                #Initialize strategy
                sim_strat = strat(**strat_params)
                sim_strat.initialize_model(G)
                
                #Initialize even alloc strategy
                even_strat = EvenAlloc()
                even_strat.initialize_model(G_even)

                sim_g, sim_utils = run_simulation(G, sim_strat)
                
                sim_g_even, sim_utils_even = run_simulation(G_even, even_strat)

                #Get global social welfare at end of simulation normalized by size
                utils.append(sum(sim_utils[-1]) / len(G.vertices))
                eutils.append(sum(sim_utils_even[-1]) / len(G_even.vertices))
            social_utils.append((p, utils))
            even_utils.append((p, eutils))

        plot_util_comp((social_utils, even_utils), r, regularity, graph_diam)

## Evenly distributed resources

In [6]:
comp_strat(num_vertices, regularity, create_rl,
        MWU, {'lrate' : lrate})

## Powerlaw distributed resources

In [7]:
comp_strat(num_vertices, regularity, create_rl,
         MWU, {'lrate' : lrate}, plaw_resources=True)